In [2]:
import tensorflow as tf

In [51]:
tf.__version__

'2.14.0'

In [52]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [2]:
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
import keras_nlp
import tensorflow_hub as hub
import numpy as np

In [4]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "1"

In [57]:
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tt247gpu2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df1 = pd.read_csv('./data/goemotions1.csv')
df2 = pd.read_csv('./data/goemotions2.csv')
df3 = pd.read_csv('./data/goemotions3.csv')

In [6]:
list(df1.columns)

['text',
 'id',
 'author',
 'subreddit',
 'link_id',
 'parent_id',
 'created_utc',
 'rater_id',
 'example_very_unclear',
 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [7]:
df1 = df1.loc[ : , ['text', 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']]

In [8]:
df2 = df2.loc[ : , ['text', 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']]

In [9]:
df3 = df3.loc[ : , ['text', 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']]

In [10]:
df1.shape

(70000, 29)

In [11]:
total_df = pd.concat([df1, df2, df3])

In [12]:
total_df.shape

(211225, 29)

In [13]:
total_df = total_df.dropna()

In [14]:
total_df.shape

(211225, 29)

In [23]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [24]:
total_df['text'][10:16]

10    I have, and now that you mention it, I think t...
11    I wanted to downvote this, but it's not your f...
12                                BUT IT'S HER TURN! /s
13                                         That is odd.
14                                    Build a wall? /jk
15    I appreciate it, that's good to know. I hope I...
Name: text, dtype: object

In [25]:
total_df['text'] = total_df['text'].apply(clean_text)
total_df['text'] = total_df['text'].str.replace('\d+', '')

In [26]:
total_df['text'][10:16]

10              mention think thats triggered nostalgia
11                          wanted downvote fault homie
12                                                 turn
13                                                  odd
14                                        build wall jk
15    appreciate thats good know hope ill apply know...
Name: text, dtype: object

In [73]:
MAX_NB_WORDS = 50000

MAX_SEQUENCE_LENGTH = 250

EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(total_df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 32388 unique tokens.


In [24]:
X = tokenizer.texts_to_sequences(total_df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (211225, 250)


In [15]:
total_df.columns

Index(['text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval',
       'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
       'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride',
       'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [16]:
Y = total_df.loc[ : , ['admiration', 'amusement', 'anger', 'annoyance', 'approval',
       'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
       'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride',
       'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']].astype(np.float32).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (211225, 28)


In [17]:
# Replacing X in train_test_split with total_df['text'] since we are now fine-tuning, model will preprocess
# text in a preset algorithm
X_train, X_test, Y_train, Y_test = train_test_split(total_df['text'],Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(190102,) (190102, 28)
(21123,) (21123, 28)


In [18]:
total_df['text'][:1]

0    That game hurt.
Name: text, dtype: object

In [19]:
Y[:1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
input = Input(shape=(MAX_SEQUENCE_LENGTH, ))
x = Embedding(MAX_NB_WORDS, EMBEDDING_DIM)(input)
x = SpatialDropout1D(0.2)(x)
x = LSTM(100, dropout=0.2, recurrent_dropout=0.2)(x)
output = Dense(28, activation='softmax')(x)
model = tf.keras.Model(inputs=input, outputs=output)

In [47]:
class My_CE_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):        
        log_y_pred = tf.math.log(y_pred)
        element_wise = -tf.math.multiply_no_nan(x=log_y_pred, y=y_true)
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

In [48]:
# Sparse categorical cross entropy has an internal error
# therefore, defined my own custom implementation of cross entropy
model.compile(loss=My_CE_Loss(), optimizer='adam', metrics=['accuracy'])

In [49]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 250)]             0         
                                                                 
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (Spatial  (None, 250, 100)          0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 28)                2828      
                                                                 
Total params: 5083228 (19.39 MB)
Trainable params: 5083228 (19.39 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [50]:
EPOCHS=10
BATCH_SIZE = 64

In [22]:
Y_train1 = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test1 = tf.convert_to_tensor(Y_test, dtype=tf.float32)

In [52]:
history = model.fit(X_train, Y_train1, epochs=EPOCHS, batch_size=BATCH_SIZE,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
2674/2674 [==============================] - 2128s 795ms/step - loss: 2.8721 - accuracy: 0.3641 - val_loss: 2.6321 - val_accuracy: 0.3964
Epoch 2/10
 618/2674 [=====>........................] - ETA: 27:44 - loss: 2.5049 - accuracy: 0.4167

KeyboardInterrupt: 

In [22]:
# Since training a model from scratch is taking a lot of compute and time
# Now setting up a preprocess-encode-decode-output pipeline for fine-tuning
preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder = hub.KerasLayer(
   'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/2',
   trainable=True)

In [23]:
txt = tf.keras.layers.Input(shape=(), dtype=tf.string)
x = preprocessor(txt)
x = encoder(x)['pooled_output']
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(Y_train1.shape[1], activation='softmax')(x)
model = tf.keras.Model(inputs=[txt], outputs=x)

In [5]:
import urllib.request
emotions = urllib.request.urlopen(
   'https://raw.githubusercontent.com/google-research/google-research'
   '/master/goemotions/data/emotions.txt').read().decode('utf8').split('\n')

In [6]:
def preprocess_dataset(split, batch_size=128):
 def one_hot_encode(x):
   vec = tf.stack([x[emotion] for emotion in emotions], 0)
   return x['comment_text'], tf.cast(vec, tf.uint8)
     
 ds = tfds.load('goemotions', split=split)
 ds = ds.map(one_hot_encode, num_parallel_calls=tf.data.AUTOTUNE)
 ds = ds.shuffle(buffer_size=batch_size * 10)
 ds = ds.batch(batch_size, drop_remainder=False)
 ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

 return ds
    
ds_splits = ['train', 'test', 'validation']
datasets = {split: preprocess_dataset(split) for split in ds_splits}

In [ ]:
## For augmenting data
# aug_spelling = naw.SpellingAug()
# aug_random = naw.RandomWordAug(action='swap')

# def iter_augmented_data(ds):
#  for x, y in iter(ds.unbatch()):
#    x = x.numpy().decode('utf8')
#    # Original text.
#    yield x, y
#    # Replace a random word by misspelling.
#    for x_aug in aug_spelling.augment(x, n=2):
#      yield x_aug, y
#    # Swap two words in input at random.
#    for x_aug in aug_random.augment(x, n=2):
#      yield x_aug, y

# gen_func = lambda: iter_augmented_data(datasets['train'])
# ds_train = tf.data.Dataset.from_generator(
#    gen_func, output_types=(tf.string, tf.uint8))
# ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)

In [32]:
# Since the tensorflow_hub BERT model experiences an internal data type issue
# on both intel and GPU optimized training, we will be using DistilBERT 
# Also, DistilBERT provides faster inference times

# Giving higher sequence lengths than what the original DistilBERT 
# fine-tuning recommended since we will be taking stories of patients into account
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(
    "distil_bert_base_en_uncased",
    sequence_length=128,
)
encoder = keras_nlp.models.DistilBertBackbone.from_preset(
    "distil_bert_base_en_uncased"
)

In [33]:
encoder.trainable = True

In [34]:
txt = tf.keras.layers.Input(shape=(), dtype=tf.string)
x = preprocessor(txt)
x = encoder(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(28, activation='softmax')(x)
model = tf.keras.Model(inputs=[txt], outputs=x)

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
metrics = [tf.keras.metrics.CategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer, loss, metrics=metrics)

In [36]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 distil_bert_preprocessor_5  {'token_ids': (None, 128),   0         ['input_5[0][0]']             
  (DistilBertPreprocessor)    'padding_mask': (None, 12                                           
                             8)}                                                                  
                                                                                                  
 distil_bert_backbone (Dist  (None, None, 768)            6636288   ['distil_bert_preprocessor_5[0
 ilBertBackbone)                                          0         ][0]',                  

In [37]:
BATCH_SIZE = 128
EPOCHS = 5
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./tmp/checkpoint',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs/DistilBERT')

In [38]:
history = model.fit(datasets['train'], validation_data=datasets['validation'], 
                    epochs=EPOCHS, verbose=2, 
                    callbacks=[model_checkpoint_callback, tensorboard_callback])

Epoch 1/5
340/340 - 197s - loss: 2.0782 - accuracy: 0.5166 - val_loss: 1.9009 - val_accuracy: 0.5534 - 197s/epoch - 578ms/step
Epoch 2/5
340/340 - 180s - loss: 1.6983 - accuracy: 0.5908 - val_loss: 1.9168 - val_accuracy: 0.5573 - 180s/epoch - 528ms/step
Epoch 3/5
340/340 - 180s - loss: 1.4581 - accuracy: 0.6478 - val_loss: 2.1907 - val_accuracy: 0.5470 - 180s/epoch - 528ms/step
Epoch 4/5
340/340 - 180s - loss: 1.2226 - accuracy: 0.7153 - val_loss: 2.4219 - val_accuracy: 0.5487 - 180s/epoch - 528ms/step
Epoch 5/5
340/340 - 180s - loss: 1.0372 - accuracy: 0.7790 - val_loss: 2.8079 - val_accuracy: 0.5365 - 180s/epoch - 528ms/step


In [40]:
model.save_weights('./checkpoints/DistilBERT_GoEmotions')

In [41]:
output = model.predict(["I am feeling down because of a death."])

1/1 [==============================] - 1s 1s/step


In [42]:
np.argmax(output[0][:-1])

16

In [ ]:
preprocessor(X_train[:3])